<a href="https://colab.research.google.com/github/OlegV12/NaturalLangProcessing/blob/Lesson_16/chatbot_new.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install pytz

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import torch
import json 
import requests
import time
import urllib 
import logging
import signal
import sys
from babel.dates import format_date, format_datetime, format_time
import datetime
from datetime import datetime
import babel
import os
from collections import defaultdict
from datetime import datetime
from urllib.request import urlretrieve, urljoin
# from urlparse import urljoin
from zipfile  import ZipFile

import pytz # pip install pytz


In [5]:
geonames_url = 'http://download.geonames.org/export/dump/'
basename = 'cities15000' # all cities with a population > 15000 or capitals
filename = basename + '.zip'

# get file
if not os.path.exists(filename):
    urlretrieve(urljoin(geonames_url, filename), filename)

# parse it
city2tz = defaultdict(set)
with ZipFile(filename) as zf, zf.open(basename + '.txt') as file:
    for line in file:
        fields = line.split(b'\t')
        if fields: # geoname table http://download.geonames.org/export/dump/
            name, asciiname, alternatenames = fields[1:4]
            timezone = fields[-2].decode('utf-8').strip()
            if timezone:
                for city in [name, asciiname] + alternatenames.split(b','):
                    city = city.decode('utf-8').strip()
                    if city:
                        city2tz[city].add(timezone)

# print("Number of available city names (with aliases): %d" % len(city2tz))

In [6]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [7]:
with open('/gdrive/My Drive/Colab Notebooks/NLP/token.txt', 'r') as f:
  token = f.read()
  token = token.replace('\n', '')

In [8]:
with open('/gdrive/My Drive/Colab Notebooks/NLP/weather_token.txt', 'r') as f:
  w_token = f.read()
  w_token = w_token.replace('\n', '')

In [10]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("deepparag/Aeona")
model = AutoModelWithLMHead.from_pretrained("deepparag/Aeona")


def get_response(text):

    step = 0
    new_user_input_ids = tokenizer.encode(text + tokenizer.eos_token, return_tensors='pt')

    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,  
        no_repeat_ngram_size=4,       
        do_sample=True, 
        top_k=100, 
        top_p=0.7,
        temperature=0.7

    )
    step += 1

    return (tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True))




/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:1002: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [9]:

!pip uninstall python-telegram-bot
!pip cache purge
!pip install python-telegram-bot

Found existing installation: python-telegram-bot 13.13
Uninstalling python-telegram-bot-13.13:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/python_telegram_bot-13.13.dist-info/*
    /usr/local/lib/python3.7/dist-packages/telegram/*
Proceed (y/n)? y
  Successfully uninstalled python-telegram-bot-13.13
Files removed: 2
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 513 kB 5.0 MB/s 


In [15]:
URL_OWM = "http://api.openweathermap.org/data/2.5/weather?appid={}&units=metric".format(w_token)

In [16]:
# Query OWM for the weather for place or coords
def makeRequest(url):

    r = requests.get(url)
    resp = json.loads(r.content.decode("utf8"))
    return resp



def getWeather(place):

        url = URL_OWM + "&q={}".format(place)

        js = makeRequest(url)

        return f"{js['name']}, temperature: {js['main']['temp']}, humidity: {js['main']['humidity']}, clouds: {js['weather'][0]['description']}"


In [17]:
from telegram.replykeyboardmarkup import ReplyKeyboardMarkup
from telegram.keyboardbutton import KeyboardButton
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters
from telegram import Update
from telegram import *
from telegram.ext import * 
from requests import *



updater = Updater(token, 
                  use_context=True)
  
# Get the dispatcher to register handlers
dp = updater.dispatcher
condition = 0
time_button = "Time"
weather_button = "Weather"


def startCommand(update: Update, context: CallbackContext):
    buttons = [[KeyboardButton(time_button)], [KeyboardButton(weather_button)]]
    context.bot.send_message(chat_id=update.effective_chat.id, text="Welcome!", 
                             reply_markup=ReplyKeyboardMarkup(buttons))


def response(update, context):
    update.message.reply_text((update.message.text))


def messageHandler(update, context):

    global condition

    if condition == 'time':
      context.bot.send_message(chat_id=update.effective_chat.id, 
                               text=babel.dates.format_datetime(datetime.utcnow(), 
                               locale='en', 
                               tzinfo=list(city2tz[update.message.text])[0]))  
      condition = 0
      return


    if condition == 'weather':
      context.bot.send_message(chat_id=update.effective_chat.id, 
                               text=getWeather(update.message.text))
      condition = 0
      return


    if time_button == update.message.text:
      context.bot.send_message(chat_id=update.effective_chat.id, 
                               text="Please name location")
      condition = 'time'


    elif weather_button == update.message.text:
      context.bot.send_message(chat_id=update.effective_chat.id, 
                               text="Please name location")
      condition = 'weather'


    else:
      if condition != 1:
        update.message.reply_text(get_response(update.message.text))
    

dp.add_handler(CommandHandler("start", startCommand))
dp.add_handler(MessageHandler(Filters.text, messageHandler))


updater.start_polling()
updater.idle()